<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220412_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers
# !pip install git+https://github.com/ssut/py-hanspell.git

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl (149.9 MB)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
# from hanspell import spell_checker

- TPU 사용을 위한 accelerate

In [4]:
from accelerate import Accelerator

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
df = df.fillna('')
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스


# 토크나이저, 함수

In [7]:
def remove_something(text:str):
  return text.replace('&',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

df['text'] = df['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

df['text'] = df['text'].map(remove_something)

In [8]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [9]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [10]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [11]:
df = pd.read_csv('/content/drive/MyDrive/220412_inference/train1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220412_inference/train'+str(i)+'.csv')
  df = pd.concat([df,temp], axis=0)

  0%|          | 0/9 [00:00<?, ?it/s]

In [12]:
df = df.reset_index(drop=True)

In [13]:
test = test.fillna('')
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test = test[['text', 'target']]

def remove_something(text:str):
  return text.replace('&',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace('.',' ')

test['text'] = test['text'].map(remove_something)

def remove_something(text:str):
  return text.replace(',',' ')

test['text'] = test['text'].map(remove_something)

In [14]:
test = pd.read_csv('/content/drive/MyDrive/220412_inference/test1.csv')

for i in tqdm(range(2, 11)):
  temp = pd.read_csv('/content/drive/MyDrive/220412_inference/test'+str(i)+'.csv')
  test = pd.concat([test,temp], axis=0)

  0%|          | 0/9 [00:00<?, ?it/s]

In [15]:
test = test.reset_index(drop=True)
test_original = test

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [16]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220412_inference/model'+str(fold), save_function=accelerator.save)

## digit 별 학습

In [ ]:
#train
train = df[['text']]
train['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)

my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
my_dict['  '] = len(my_dict)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )

for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  my_file = Path('/content/drive/MyDrive/220412_inference/model'+str(fold))
  if my_file.is_dir():
    continue
  print(f'FOLD {fold}')
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  train_label = train['target']
  train_dataset = BERTDataset(tokenized_train, train_label)
  notebook_launcher(training_function)

#test
test = test_original
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
test_label = test['target'].map(target_to_num)
tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)
dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

for fold in range(5):
  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220412_inference/model'+str(fold), config=config)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)
  output_pred = []
  output_prob = []
  model, dataloader= accelerator.prepare(model, dataloader)
  model.eval()
  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []
      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      return str_label

  answer = num_to_label(pred_answer)
  df_prob = pd.DataFrame(output_prob)
  df_prob.to_csv('/content/drive/MyDrive/220412_inference/prob_pred'+str(fold)+'.csv', index=False)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

...

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

epoch 1 train acc 0.70571
epoch 1 train acc 0.70764
epoch 1 train acc 0.70663
epoch 1 train acc 0.70267
epoch 1 train acc 0.70665
epoch 1 train acc 0.70686
epoch 1 train acc 0.70526
epoch 1 train acc 0.70574
epoch 1 test acc 0.8677669437340153
epoch 1 test acc 0.8672074808184144
epoch 1 test acc 0.8696451406649617
epoch 1 test acc 0.8743206521739131
epoch 1 test acc 0.8633711636828645
epoch 1 test acc 0.8696851023017903
epoch 1 test acc 0.8656889386189258
epoch 1 test acc 0.8703644501278772


## Voting

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220412_inference/prob_pred0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220412_inference/prob_pred1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220412_inference/prob_pred2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220412_inference/prob_pred3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220412_inference/prob_pred4.csv')

In [ ]:
pred_1 = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test_1 = pd.concat([test, pred_1], axis=1)
test_1.head(3)

In [ ]:
train = df[['text']]
train['target_1'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)

my_dict1 = {}
for i in range(len(train['target_1'].unique())):
  my_dict1[train['target_1'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict1[target]

In [ ]:
def num_to_label1(label):
    label_dict = label_dict = {v: k for k, v in my_dict1.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label1(np.argmax(np.array(pred_1), axis=-1))

test_1['soft_1'] = pd.DataFrame(answer)[1]

# 제출하기

In [ ]:
def split_target(text:str):
  return text.split(' ')[0]

submission['digit_1'] = test_1['soft_1'].map(split_target)

def split_target(text:str):
  return text.split(' ')[1]

submission['digit_2'] = test_1['soft_1'].map(split_target)

def split_target(text:str):
  return text.split(' ')[2]

submission['digit_3'] = test_1['soft_1'].map(split_target)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220412_inference/submission.csv', index=False)